## Simplifying data with the singular value decomposition

- Pros: Simplifies data, removes noise, may improve algorithm results.
- Cons: Transformed data may be difficult to understand.
- Works with: Numeric values.

In [1]:
import numpy as np

### Experiment 1: Toy dataset

In [2]:
def loadExData():
    return np.array(
             [[0, 0, 0, 2, 2],
             [0, 0, 0, 3, 3],
             [0, 0, 0, 1, 1],
             [1, 1, 1, 0, 0],
             [2, 2, 2, 0, 0],
             [5, 5, 5, 0, 0],
             [1, 1, 1, 0, 0]])

In [3]:
Data = loadExData()
U, Sigma, VT = np.linalg.svd(Data)
print(Sigma)

[9.64365076e+00 5.29150262e+00 6.49628424e-16 1.43063514e-16
 2.79192092e-17]


In [4]:
Sig3 = np.diag(Sigma[:3])
with np.printoptions(precision=2):
    print(np.dot(np.dot(U[:, :3], Sig3), VT[:3]))

[[ 3.48e-16 -6.25e-16 -1.12e-15  2.00e+00  2.00e+00]
 [ 5.29e-16 -4.09e-16 -1.21e-16  3.00e+00  3.00e+00]
 [ 1.63e-16 -1.40e-16 -2.29e-17  1.00e+00  1.00e+00]
 [ 1.00e+00  1.00e+00  1.00e+00 -1.70e-16 -1.70e-16]
 [ 2.00e+00  2.00e+00  2.00e+00 -2.45e-16 -2.45e-16]
 [ 5.00e+00  5.00e+00  5.00e+00 -5.55e-18 -5.55e-18]
 [ 1.00e+00  1.00e+00  1.00e+00 -1.23e-16 -1.23e-16]]


### Experiment 2: Recommendation system

In [5]:
# Similarity measures
def ecludSim(inA, inB):
    return 1.0 / (1.0 + np.linalg.norm(inA - inB))


def pearsSim(inA, inB):
    # if len(inA) < 3:
    #     return 1.0
    if np.var(inA) == 0 or np.var(inB) == 0:
        return 1.0
    return 0.5 + 0.5 * np.corrcoef(inA, inB)[0][1]


def cosSim(inA, inB):
    num = np.dot(inA, inB)
    denom = np.linalg.norm(inA) * np.linalg.norm(inB)
    return 0.5 + 0.5 * (num / denom)

In [6]:
# Item-based recommendation engine
def standEst(dataMat, user, simMeas, item):
    simTotal, ratSimTotal = 0.0, 0.0
    for j in range(dataMat.shape[1]):
        userRating = dataMat[user, j]
        if userRating == 0:
            continue
        # Find users who rated both items
        overLap = np.nonzero(np.logical_and(dataMat[:, item] > 0,
                                            dataMat[:, j] > 0))[0]
        if len(overLap) == 0:
            similarity = 0
        else:
            similarity = simMeas(dataMat[overLap, item], dataMat[overLap, j])
        # print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0:
        return 0
    else:
        return ratSimTotal / simTotal


def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    # Find unrated items
    unratedItems = np.nonzero(dataMat[user] == 0)[0]
    if len(unratedItems) == 0:
        return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    # Return top N unrated items
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

In [7]:
myMat = loadExData()
myMat[0, 1] = myMat[0, 0] = myMat[1, 0] = myMat[2, 0] = 4
myMat[3, 3] = 2
print(myMat)

[[4 4 0 2 2]
 [4 0 0 3 3]
 [4 0 0 1 1]
 [1 1 1 2 0]
 [2 2 2 0 0]
 [5 5 5 0 0]
 [1 1 1 0 0]]


In [8]:
recommend(myMat, 2)

[(2, 2.5), (1, 2.0243290220056256)]

In [9]:
recommend(myMat, 2, simMeas=ecludSim)

[(2, 3.0), (1, 2.8266504712098603)]

In [10]:
recommend(myMat, 2, simMeas=pearsSim)

[(2, 2.5), (1, 2.0)]

In [11]:
# Rating estimation by using the SVD
def svdEst(dataMat, user, simMeas, item):
    simTotal, ratSimTotal = 0.0, 0.0
    U, Sigma, VT = np.linalg.svd(dataMat)
    # Create diagonal matrix
    # Sig3 = np.diag(Sigma[:3])
    # Create transformed items
    # xformedItems = np.dot(np.dot(dataMat.T, U[:, :3]), np.linalg.inv(Sig3))
    xformedItems = np.dot(dataMat.T, U[:, :3])
    for j in range(dataMat.shape[1]):
        userRating = dataMat[user, j]
        if userRating == 0:
            continue
        similarity = simMeas(xformedItems[item], xformedItems[j])
        # print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0:
        return 0
    else:
        return ratSimTotal / simTotal

In [12]:
def loadExData2():
    return np.array(
             [[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
             [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
             [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
             [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
             [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
             [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
             [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
             [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
             [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
             [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
             [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]])

In [13]:
myMat = loadExData2()

In [14]:
U, Sigma, VT = np.linalg.svd(myMat)
Sig2 = Sigma ** 2

In [15]:
print(sum(Sig2) * 0.9)
print(sum(Sig2[:2]))
print(sum(Sig2[:3]))

487.7999999999995
378.8295595113579
500.5002891275793


In [16]:
recommend(myMat, 1, estMethod=svdEst)

[(6, 3.3330507283671458), (9, 3.3315053278149733), (4, 3.3313983687864637)]

In [17]:
recommend(myMat, 1, estMethod=svdEst, simMeas=pearsSim)

[(6, 3.330644816334907), (9, 3.327344778372606), (4, 3.327175498702263)]

### Experiment 3: Image compression

In [18]:
# Image-compression functions
def printMat(inMat, thresh=0.8):
    for i in range(32):
        print((inMat[i] > thresh).astype(int))


def imgCompress(numSV=3, thresh=0.8):
    myl = []
    for line in open('0_5.txt').readlines():
        newRow = []
        for i in range(32):
            newRow.append(int(line[i]))
        myl.append(newRow)
    myMat = np.array(myl)
    print("****original matrix******")
    printMat(myMat, thresh)
    U, Sigma, VT = np.linalg.svd(myMat)
    SigRecon = np.diag(Sigma[:numSV])
    reconMat = np.dot(np.dot(U[:, :numSV], SigRecon), VT[:numSV])
    print("****reconstructed matrix using %d singular values******" % numSV)
    printMat(reconMat, thresh)

In [19]:
imgCompress(2)

****original matrix******
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
[0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1